In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim

# Überprüfe, ob PyTorch verfügbar ist
print("PyTorch Version:", torch.__version__)
print("CUDA verfügbar:", torch.cuda.is_available())
print("CUDA-Version:", torch.version.cuda)
print("Aktuelle GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Keine GPU")



In [ ]:
'''
# Transformations für den Datensatz
transform = transforms.Compose([
    transforms.ToTensor()  # Wandelt die Bilder von PIL-Images oder NumPy-Arrays in Tensors um.
                            # Die Werte der Pixel werden automatisch von [0, 255] auf den Bereich [0, 1] skaliert
    #, transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 laden
batch_size = 64
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)





# Lade das vortrainierte ResNet-18 Modell
model = resnet18(pretrained=True)

# Passe die finale Schicht für CIFAR-10 an, da ResNet-18 für ImageNet mit 1000 Klassen trainiert wurde
model.fc = nn.Linear(512, 10)  # CIFAR-10 hat 10 Klassen

# Setze das Modell in den Evaluierungsmodus
model.eval()

# Bestimme das device (GPU oder CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("used  device: ", device)

# Verschiebe das Modell auf das richtige device (CPU oder GPU)
model = model.to(device)
'''


In [ ]:
# Daten-Transformationen
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Zufälliger Zuschnitt
    transforms.RandomHorizontalFlip(),     # Spiegeln
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Daten laden
train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck']

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
# Lade das vortrainierte ResNet-18 Modell
model = resnet18(pretrained=True)

# Passe die finale Schicht an
model.fc = nn.Linear(512, 10)  # CIFAR-10 hat 10 Klassen

# Device setzen
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Loss-Funktion
criterion = nn.CrossEntropyLoss()

# Optimierer
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Learning Rate Scheduler (optional, für bessere Ergebnisse)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Trainingsschleife

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()  # Setze das Modell in den Trainingsmodus
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Gradienten zurücksetzen
        outputs = model(inputs)  # Vorhersagen
        loss = criterion(outputs, labels)  # Loss berechnen
        loss.backward()  # Backpropagation
        optimizer.step()  # Optimierung

        running_loss += loss.item()

    scheduler.step()  # Learning Rate anpassen (falls Scheduler verwendet wird)

    # Trainingsgenauigkeit berechnen
    model.eval()  # Setze das Modell in den Evaluierungsmodus
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
    train_accuracy = 100 * correct_train / total_train

    # Validierungsgenauigkeit berechnen
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    val_accuracy = 100 * correct_val / total_val

    # Ausgabe der Ergebnisse
    print(f'Epoch {epoch+1}/{epochs}')
    print(f'    Loss: {running_loss / len(train_loader):.4f}')
    print(f'    Train Accuracy: {train_accuracy:.2f}%')
    print(f'    Validation Accuracy: {val_accuracy:.2f}%')


# Speichern und Laden des Modells

In [ ]:
# Speichern
torch.save(model.state_dict(), 'resnet18_cifar10.pth')

# Laden
model.load_state_dict(torch.load('resnet18_cifar10.pth'))
model.eval()

In [ ]:
# Accuracy berechnen
correct = 0
total = 0

# Deaktiviere Gradient-Berechnungen für die Inferenz
with torch.no_grad():
    # Durchlaufe den Testdatensatz
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Modellvorhersagen durchführen
        outputs = model(images)

        # Bestimme die Vorhersage (Index mit der höchsten Wahrscheinlichkeit)
        _, predicted = torch.max(outputs, 1)

        # Anzahl der richtigen Vorhersagen zählen
        total += labels.size(0)  # Die Anzahl der Bilder im Batch
        correct += (predicted == labels).sum().item()  # Die Anzahl der richtigen Vorhersagen

# Berechne die Accuracy
accuracy = 100 * correct / total
print(f'Accuracy des Modells auf den Testdaten: {accuracy:.2f}%')

# Wir beginnen mit einer Fast Gradient Sign Method (FGSM) Attacke:

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    """
    Führt eine FGSM-Attacke aus.
    :param image: Eingabebild
    :param epsilon: Perturbationsstärke
    :param data_grad: Gradienten der Eingabe
    :return: Angegriffenes Bild
    """
    sign_data_grad = data_grad.sign()  # Gradienten in Vorzeichen umwandeln
    perturbed_image = image + epsilon * sign_data_grad  # Perturbation hinzufügen
    perturbed_image = torch.clamp(perturbed_image, -1, 1)  # Pixelwerte begrenzen
    return perturbed_image


In [ ]:
epsilon = 8/255  # to normalized Data, plots of various epsilons
correct = 0
adv_examples_fgsm = []

for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    data.requires_grad = True  # Eingabebild für Gradienten vorbereiten

    # Vorhersage und Verlust berechnen
    output = model(data)
    loss = nn.CrossEntropyLoss()(output, target)

    # Gradienten berechnen
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data

    # FGSM-Attacke durchführen
    perturbed_data = fgsm_attack(data, epsilon, data_grad)

    # Neue Vorhersage
    output = model(perturbed_data)
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()

    # Speichere 5 Beispiele
    if len(adv_examples_fgsm) < 5:
        adv_ex = perturbed_data[0].detach().cpu().numpy()
        adv_examples_fgsm.append((epsilon, pred[0].item(), target[0].item(), adv_ex))

# Genauigkeit berechnen
final_acc = correct / len(test_loader.dataset)
print(f"Test Accuracy mit Fast Gradient Sign Method (FGSM) Attacke = {final_acc * 100:.2f}%    (epsilon: {epsilon})")


In [ ]:
def imshow(img):
    # Falls das Bild ein PyTorch Tensor ist, normalisieren und in numpy Array umwandeln
    if isinstance(img, torch.Tensor):
        img = img / 2 + 0.5  # Normalisierung zurücksetzen
        npimg = img.numpy()
    else:
        # Falls es schon ein numpy Array ist, einfach darauf zugreifen
        npimg = img

    # Transponiere die Dimensionen von (C, H, W) zu (H, W, C)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

print("Angegriffene Bilder:")
for epsilon, pred, true, ex in adv_examples_fgsm:
    print(f"Epsilon: {epsilon}\tVorhersage: {class_names[pred]}\tEchtes Label: {class_names[true]}")
    imshow(ex)  # Zeigt das Bild an


# Jetzt führen wir eine Projected Gradient Descent (PGD) Attacke durch:

In [ ]:
def pgd_attack(model, image, label, epsilon, alpha, num_iter):
    """
    Führt eine PGD-Attacke aus.

    :param model: Das zu attackierende Modell
    :param image: Eingabebild
    :param label: Wahres Label des Bildes
    :param epsilon: Maximale Perturbation (L∞-Norm)
    :param alpha: Schrittweite der Perturbation
    :param num_iter: Anzahl der Iterationen
    :return: Angegriffenes Bild
    """
    # Stelle sicher, dass das Bild gradient-fähig ist
    perturbed_image = image.clone().detach().requires_grad_(True)

    for _ in range(num_iter):
        # Setze die Gradienten auf 0
        model.zero_grad()

        # Berechne die Vorhersage und den Verlust
        output = model(perturbed_image)
        loss = nn.CrossEntropyLoss()(output, label)

        # Berechne die Gradienten
        loss.backward()
        data_grad = perturbed_image.grad.data

        # Berechne die neue Perturbation (ähnlich wie FGSM, aber iterativ)
        perturbed_image = perturbed_image + alpha * data_grad.sign()

        # Projektion der Perturbation in den erlaubten Epsilon-Bereich
        perturbation = torch.clamp(perturbed_image - image, -epsilon, epsilon)
        perturbed_image = torch.clamp(image + perturbation, -1, 1).detach()
        perturbed_image.requires_grad = True  # Gradient wieder aktivieren

    return perturbed_image


In [ ]:
# Parameter der PGD-Attacke
epsilon = 8 / 255  # Maximale Perturbation
alpha = 2 / 255    # Schrittgröße pro Iteration
num_iter = 10      # Anzahl der Iterationen

correct = 0
adv_examples_pgd = []

for data, target in test_loader:
    # Lade Bilder und Labels auf das richtige Gerät
    data, target = data.to(device), target.to(device)

    # Wende die PGD-Attacke auf jedes Bild im Batch an
    perturbed_data = pgd_attack(model, data, target, epsilon, alpha, num_iter)

    # Vorhersage mit dem Modell auf die angegriffenen Bilder
    output = model(perturbed_data)
    pred = output.max(1, keepdim=True)[1]  # Klasse mit der höchsten Wahrscheinlichkeit
    correct += pred.eq(target.view_as(pred)).sum().item()  # Zähle korrekte Vorhersagen

    # Speichere 10 Beispiele
    if len(adv_examples_pgd) < 10:
        for i in range(min(len(data), 10 - len(adv_examples_pgd))):
            adv_ex = perturbed_data[i].detach().cpu().numpy()
            adv_examples_pgd.append((epsilon, pred[i].item(), target[i].item(), adv_ex))

# Genauigkeit berechnen
final_acc = correct / len(test_loader.dataset)
print(f"Test Accuracy mit Projected Gradient Descent (PGD) Attacke = {final_acc * 100:.2f}%    (epsilon: {epsilon})")


In [ ]:
def imshow(img):
    # Falls das Bild ein PyTorch Tensor ist, normalisieren und in numpy Array umwandeln
    if isinstance(img, torch.Tensor):
        img = img / 2 + 0.5  # Normalisierung zurücksetzen
        npimg = img.numpy()
    else:
        # Falls es schon ein numpy Array ist, einfach darauf zugreifen
        npimg = img

    # Transponiere die Dimensionen von (C, H, W) zu (H, W, C)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


print("Angegriffene Bilder:")
for epsilon, pred, true, ex in adv_examples_pgd:
    print(f"Epsilon: {epsilon}\tVorhersage: {class_names[pred]}\tEchtes Label: {class_names[true]}")
    imshow(ex)  # Zeigt das Bild an
